<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/Threads_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo Serial

In [1]:
%%writefile ejemplo1_nothread.c

#include <stdio.h>

void f1(int *);
void f2(int *);
void merge(int, int);

int r1 = 0, r2 = 0;

extern int
main(void){
  f1(&r1);
  f2(&r2);
  merge(r1, r2);
  return 0;
}

void f1(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f1  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void f2(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f2  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void merge(int one_times, int another_times)
{
 int total;

 total = one_times + another_times;
 printf("merge: f1 %d, f2 %d, total %d\n",one_times, another_times, total);
}


Writing ejemplo1_nothread.c


In [2]:
!gcc ejemplo1_nothread.c -o ejemplo1_nothread -lpthread

In [4]:
!./ejemplo1_nothread

f1  iter: 0
f1  iter: 1
f1  iter: 2
f1  iter: 3
f2  iter: 0
f2  iter: 1
f2  iter: 2
f2  iter: 3
merge: f1 4, f2 4, total 8


# Ejemplo fork

In [5]:
%%writefile ejemplo1_fork.c

#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/ipc.h>
#include <sys/shm.h>
#include <sys/wait.h>



void f1(int *);
void f2(int *);
void merge(int, int);

int   shared_mem_id;
int   *shared_mem_ptr;
int   *r1p;
int   *r2p;

extern int
main(void){

  pid_t  child1_pid, child2_pid;
  int  status;

 /* inicializamos el segmento de memoria compartida */
  shared_mem_id = shmget(IPC_PRIVATE, 2*sizeof(int), 0660);
  shared_mem_ptr = (int *)shmat(shared_mem_id, (void *)0, 0);
  r1p = shared_mem_ptr;
  r2p = (shared_mem_ptr + 1);

  *r1p = 0;
  *r2p = 0;
  
  /* primer hijo */
  if ((child1_pid = fork()) == 0) {
    f1(r1p);
    exit(0);
  }

  /* segundo hijo */
  if ((child2_pid = fork()) == 0) {
    f2(r2p);
    exit(0);
  }

  /* el proceso padre (main) espera por los hijos */
  waitpid(child1_pid, &status, 0);
  waitpid(child2_pid, &status, 0);

  merge(*r1p, *r2p);
  return 0;
}

void f1(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f1  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void f2(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f2  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void merge(int one_times, int another_times)
{
 int total;

 total = one_times + another_times;
 printf("merge: f1 %d, f2 %d, total %d\n",one_times, another_times, total);
}


Writing ejemplo1_fork.c


In [6]:
!gcc ejemplo1_fork.c -o ejemplo1_fork -lpthread

In [9]:
!./ejemplo1_fork

f1  iter: 0
f2  iter: 0
f2  iter: 1
f1  iter: 1
f2  iter: 2
f1  iter: 2
f2  iter: 3
f1  iter: 3
merge: f1 4, f2 4, total 8


# Ejemplo Hilos

In [10]:
%%writefile ejemplo1_hilos.c

#include <stdio.h>
#include <pthread.h>

void f1(int *);
void f2(int *);
void merge(int, int);

int r1 = 0, r2 = 0;

extern int
main(void)
{
  pthread_t thread1,  thread2;

  pthread_create(&thread1,
          NULL,
          (void *) f1,
          (void *) &r1);

  pthread_create(&thread2,
          NULL,
          (void *) f2,
          (void *) &r2);

  pthread_join(thread1,  NULL);
  pthread_join(thread2,  NULL);

  merge(r1, r2);
  return 0;
}


void f1(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f1  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void f2(int *pnum_times)
{
  int i, j, x;

  for (i = 0;    i  <  4;  i++) {
    printf("f2  iter: %d\n",*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }
}

void merge(int one_times, int another_times)
{
 int total;

 total = one_times + another_times;
 printf("merge: f1 %d, f2 %d, total %d\n",one_times, another_times, total);
}

Writing ejemplo1_hilos.c


In [11]:
!gcc ejemplo1_hilos.c -o ejemplo1_hilos -lpthread

In [16]:
!./ejemplo1_hilos

f1  iter: 0
f2  iter: 0
f2  iter: 1
f1  iter: 1
f2  iter: 2
f1  iter: 2
f2  iter: 3
f1  iter: 3
merge: f1 4, f2 4, total 8


In [17]:
%%writefile ejemplo1_hilos_mutex.c

#include <stdio.h>
#include <pthread.h>

void f1(int *);
void f2(int *);
void merge(int, int);

int r1 = 0, r2 = 0;

int repeat=5;

pthread_mutex_t repeat_mutex=PTHREAD_MUTEX_INITIALIZER;

extern int
main(void)
{
  pthread_t thread1,  thread2;

  pthread_create(&thread1,
          NULL,
          (void *) f1,
          (void *) &r1);

  pthread_create(&thread2,
          NULL,
          (void *) f2,
          (void *) &r2);

  pthread_join(thread1,  NULL);
  pthread_join(thread2,  NULL);

  merge(r1, r2);
  return 0;
}


void f1(int *pnum_times)
{
  int i, j, x, r;

 do{
  pthread_mutex_lock(&repeat_mutex);
  r=repeat;
  repeat--;
  pthread_mutex_unlock(&repeat_mutex);

  //corremos nuestro ciclo
  for (i = 0;    i  <  4;  i++) {
    printf("f1  iter: %d repeat: %d times: %d\n",i,r,*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }

 }while(r>1);

}

void f2(int *pnum_times)
{
  int i, j, x, r;

 do{
  pthread_mutex_lock(&repeat_mutex);
  r=repeat;
  repeat--;
  pthread_mutex_unlock(&repeat_mutex);

  //corremos nuestro ciclo
  for (i = 0;    i  <  4;  i++) {
    printf("f2  iter: %d repeat: %d times: %d\n",i,r,*pnum_times);
    for (j = 0; j < 10000000; j++) x = x + i;
    (*pnum_times)++;
  }

 }while(r>1);

}

void merge(int one_times, int another_times)
{
 int total;

 total = one_times + another_times;
 printf("merge: f1 %d, f2 %d, total %d\n",one_times, another_times, total);
}

Writing ejemplo1_hilos_mutex.c


In [18]:
!gcc ejemplo1_hilos_mutex.c -o ejemplo1_hilos_mutex -lpthread

In [21]:
!./ejemplo1_hilos_mutex

f1  iter: 0 repeat: 5 times: 0
f2  iter: 0 repeat: 4 times: 0
f1  iter: 1 repeat: 5 times: 1
f2  iter: 1 repeat: 4 times: 1
f1  iter: 2 repeat: 5 times: 2
f2  iter: 2 repeat: 4 times: 2
f1  iter: 3 repeat: 5 times: 3
f2  iter: 3 repeat: 4 times: 3
f1  iter: 0 repeat: 3 times: 4
f2  iter: 0 repeat: 2 times: 4
f1  iter: 1 repeat: 3 times: 5
f2  iter: 1 repeat: 2 times: 5
f1  iter: 2 repeat: 3 times: 6
f2  iter: 2 repeat: 2 times: 6
f1  iter: 3 repeat: 3 times: 7
f2  iter: 3 repeat: 2 times: 7
f1  iter: 0 repeat: 1 times: 8
f2  iter: 0 repeat: 0 times: 8
f1  iter: 1 repeat: 1 times: 9
f2  iter: 1 repeat: 0 times: 9
f1  iter: 2 repeat: 1 times: 10
f2  iter: 2 repeat: 0 times: 10
f1  iter: 3 repeat: 1 times: 11
f2  iter: 3 repeat: 0 times: 11
merge: f1 12, f2 12, total 24
